In [2]:
import pandas as pd
import numpy as np
import requests
import json
import time
from IPython.display import clear_output

In [2]:
def get_coord_and_time(lon_cent, lat_cent, lon_el, lat_el, mode):
    SERVER_URL_MSK_DRIVING = 'http://178.62.250.186:5000/route/v1/driving/'
    SERVER_URL_MSK_FOOT = 'http://178.62.250.186:5001/route/v1/foot/'
    SERVER_URL_PETER_DRIVING = 'http://178.62.250.186:5002/route/v1/driving/'
    SERVER_URL_PETER_FOOT = 'http://178.62.250.186:5003/route/v1/foot/'
    try:
        if lat_cent > 58 and lat_el > 58:
            a = {
                'driving': SERVER_URL_PETER_DRIVING,
                'foot': SERVER_URL_PETER_FOOT,
                }[mode]
        else:
            a = {
                'driving': SERVER_URL_MSK_DRIVING,
                'foot': SERVER_URL_MSK_FOOT,
                }[mode]
    except KeyError as e:
        raise ValueError('Undefined mode: {}'.format(mode))


    b = str(lon_cent)+','+str(lat_cent)+';' + str(lon_el)+','+str(lat_el)
    c = '?annotations=duration'
    d = '&alternatives=false'
    e = '&overview=full&geometries=geojson'

    page_url =  a+b+c +d +e
    while True:
        try:
            page = requests.get(page_url)
            #print(page_url)
            break
        except:
            print('exception',end=' ')
            time.sleep(1)
    data = page.json()
    #print(data)
    coord_list = data['routes'][0]['geometry']['coordinates']
    time_list = data['routes'][0]['legs'][0]['annotation']['duration']
    return coord_list,time_list, data


In [3]:
mode = 'driving'

In [4]:
lat0,lon0 = 55.781178, 37.635220


In [5]:
df_habit = pd.read_excel('Жилые дома с числом квартир и ценой v4.xlsx')
#df_habit = pd.read_excel('offices_base_ready.xlsx')

R_lat = 0.03
R_lon = R_lat*4

filt_lon = np.abs(df_habit['lon'] - lon0) < R_lon
filt_lat = np.abs(df_habit['lat'] - lat0) < R_lat
df_habit = df_habit[filt_lon&filt_lat]

#filt_lon = np.abs(df_habit['lon'] - lon0) > R*0.75
#filt_lat = np.abs(df_habit['lat'] - lat0) > R*0.75
df_habit = df_habit[filt_lon&filt_lat]
#df_habit = df_habit.iloc[:10]

/home/kvertea/.local/lib/python3.5/site-packages/ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]


In [6]:
df_att = pd.read_excel('metro msk_spb.xlsx')
att_mult = 2
filt_lon = np.abs(df_att['lon'] - lon0) < att_mult*R_lon
filt_lat = np.abs(df_att['lat'] - lat0) < att_mult*R_lat
df_att = df_att[filt_lon&filt_lat]#.iloc[0:1]

In [7]:
coords = [
    (55.748939, 37.608223),
    (55.753433, 37.612106),
    (55.756918, 37.616398),
    (55.758982, 37.621544),
    (55.759055, 37.626222),
    (55.756360, 37.630004),
    (55.750067, 37.631677),
    (55.752032, 37.624351),
    ]
df_att = pd.DataFrame(coords, columns = ['lat','lon'])
df_att['obj_name'] = 'center'

In [8]:
print(len(df_att))
print(len(df_habit))

8
5164


In [9]:
def get_dic(link):
    for i in range(100):
        content = requests.get(link).content
        dic = json.loads(content.decode())
        if 'message' not in dic.keys():
            break
        else:
            time.sleep(1)
            clear_output()
            print(dic)
    return dic

In [10]:
df_lines = pd.DataFrame()

In [11]:
from tqdm import tqdm

In [12]:
from functools import reduce


In [13]:
area_lat_lon_rate = 2

for ind, habit_row in tqdm(df_habit.iterrows()):
    end_lat, end_lon = habit_row['lat'], habit_row['lon']

    coords = df_att[['lon','lat']].values
    

    pseudo_Rs = map(lambda x: (x[0]**2 + area_lat_lon_rate*(x[1]**2))**0.5,
                    (coords - np.array([habit_row['lon'],habit_row['lat']]))) 
    pseudo_Rs = list(pseudo_Rs)
    df_att['pseudo_R'] = pseudo_Rs
    N_atts = 6
    gb = df_att.groupby('obj_name')['pseudo_R'].min()
    if len(set(gb))>=N_atts:
        pseudo_R_boarder = df_gb.sort_values().iloc[N_atts]
        df_att_temp = df_att[df_att['pseudo_R']<pseudo_R_boarder]
    else:
        df_att_temp = df_att.copy()
    
    coords = df_att_temp[['lon','lat']].values
    
    
    link = 'http://178.62.250.186:5001/table/v1/foot/'
    link += str(end_lon) + ',' + str(end_lat)
    for coord in coords:
        link += ';' + str(coord[0]) + ',' + str(coord[1])
    link += '?sources=0'
    dic = get_dic(link)

    durations = np.array(dic['durations'][0][1:])
    min_duration = np.min(durations)
#     if min_duration > 60*15:
#         #print('too long',end= ' ')
#         continue
    att_row = df_att_temp[durations==min_duration].iloc[0]

    start_lat, start_lon = att_row['lat'], att_row['lon']
    lat_cent, lon_cent = att_row['lat'], att_row['lon']
    lat_el, lon_el  = habit_row['lat'], habit_row['lon']
    
    coords, times, data =  get_coord_and_time(lon_cent, lat_cent, lon_el, lat_el, mode)

    df_one_line = pd.DataFrame(coords, columns=['lon','lat'])
    df_line = df_one_line.iloc[:-1].rename(columns = {'lat':'lat0','lon':'lon0'})
    df_line['lat1'] = df_one_line['lat'].iloc[1:].values
    df_line['lon1'] = df_one_line['lon'].iloc[1:].values
    df_line['weight'] = habit_row['flat_num']
    df_lines = pd.concat([df_lines,df_line])

5164it [06:22, 13.49it/s]


In [14]:
np.min(durations)

4527.2

In [15]:
att_row

lat           55.7564
lon             37.63
obj_name       center
pseudo_R    0.0837125
Name: 5, dtype: object

In [16]:
len(df_one_line)

252

In [17]:
att_row['lat'], att_row['lon']

(55.75636, 37.630004)

In [18]:
habit_row['lat'], habit_row['lon']

(55.7571, 37.71371)

In [19]:
start_lat, start_lon

(55.75636, 37.630004)

In [20]:
df_lines_gb = df_lines.groupby(['lon0','lat0','lon1','lat1'])['weight'].sum()
df_lines_gb = df_lines_gb.reset_index()

In [21]:
import folium

In [22]:
from colour import Color

In [23]:
import matplotlib.pyplot as plt

In [24]:
length = len(df_lines_gb)
length1 = int(np.round(length*0.7))
length2 = length - length1
part1 = Color('Green').range_to('Yellow',length1)
part2 = Color('Yellow').range_to('Red',length2)
colors = list(part1) + list(part2)
colors = list(map(lambda x: x.get_web(),colors))

In [25]:
df_lines_gb = df_lines_gb.sort_values('weight', ascending=True)

In [26]:
df_lines_gb['color'] = colors

In [27]:
#df_lines_gb = df_lines_gb.sort_values('weight', ascending=False)
map_ = folium.Map(location = list(df_lines_gb[['lat0','lon0']].mean().values), zoom_start=12)
for i, row in tqdm(df_lines_gb.iterrows()):
    line = [
    (row['lat0'],row['lon0']),
    (row['lat1'],row['lon1'])
    ]
    folium.PolyLine(line,color = row['color']).add_to(map_)
for i, row in tqdm(df_att.iterrows()):
    folium.Circle([row['lat'],row['lon']],color = 'black').add_to(map_)

map_.save('maps/folk_paths_wide1_driving.html')

32151it [05:00, 106.96it/s]
8it [00:00, 113.30it/s]


NameError: name 'folium' is not defined